<a href="https://colab.research.google.com/github/dcorreavi/analysis-everpeak/blob/main/everpeak_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 EverPeak — Analysis


---

## 1. Importar librerías y cargar el dataset

🎯 **Objetivo:** tener el datasets listos en memoria.

Vas a:
- Importar `pandas`, `numpy`, `matplotlib`, `seaborn`.
- Cargar `everpeak_raw.csv`
- Hacer una inspección rápida de las primeras filas.



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



df = pd.read_csv("everpeak_raw.csv")


print("df:")
print(df.head())


---

## 2. Explorar el 3 dataset

🎯 **Objetivo:** entender la estructura básica de cada tabla.

Vas a:
- Revisar tipos de datos (`dtypes`).
- Revisar número de filas y columnas.
- Identificar claves principales (ej. `user_id`).

### 🔍 Hint
Usa:
- `.shape`
- `.dtypes`
- `.info()`


---

## 3. Identificar problemas de calidad de datos

🎯 **Objetivo:** encontrar columnas con:
- valores faltantes,
- sentinels (ej. `-999`, `"?"`),
- valores imposibles (fechas futuras, etc.),
- registros inconsistentes.


`users`

1. Calcular proporción de `NaN` por columna.
2. Contar cuántos `-999` hay en `age`.
3. Contar cuántos `"?"` hay en `city` u otras columnas.
4. Revisar fechas fuera de rango en `reg_date` o `date`.

`usage`

5. Revisar valores faltantes .

### 🔍 Hint
- `users.isna().mean()` te da proporción de missing.
- `(users['age'] == -999).sum()` cuenta sentinels.
- `users['city'].value_counts(dropna=False)` ayuda a ver valores raros.


## 7. Identificación y tratamiento de outliers

Los outliers son valores extremos que pueden representar:

Clientes VIP con uso legítimamente alto

Usuarios corporativos

Errores en el dato

Comportamientos atípicos que vale la pena entender

En este proyecto queremos que aprendas cómo detectarlos y luego decidas qué hacer con ellos según el contexto de negocio.


La detección se hará en tres etapas, tal como se enseña en la unidad de outliers:
visual → estadística → decisión contextual.

### 7.1 Identificación visual de outliers (boxplots)

🎯 Objetivo: usar gráficos para detectar valores extremos rápidamente.

Los boxplots permiten ver:

rango intercuartil (IQR)

valores extremos (puntos aislados)

distribución general

🔍 Hint

Usa:

sns.boxplot(x=user_profile['total_call_minutes'])


Ajusta plt.xlim() para ver mejor.

### 7.2 Identificación estadística de outliers (IQR method)

🎯 Objetivo: detectar outliers usando un criterio estadístico reproducible.

En este paso aplicarás el método IQR:

Q1 = percentil 25

Q3 = percentil 75

IQR = Q3 – Q1

Límite superior = Q3 + 1.5×IQR

🔍 Hint

Usa .quantile()

Crea la bandera:

user_profile['outlier_flag_minutes'] = user_profile['total_call_minutes'] > upper

In [ ]:
# Nuevo análisis: detectar outliers con IQR
Q1 = df['order_value'].quantile(0.25)
Q3 = df['order_value'].quantile(0.75)
IQR = Q3 - Q1
upper = Q3 + 1.5*IQR
df['outlier_flag'] = df['order_value'] > upper


### 7.3 Decidir cómo manejar los outliers (mantener, corregir o eliminar)

🎯 Objetivo: aplicar criterio de negocio para decidir el tratamiento adecuado.

En ConnectaTel, los outliers pueden significar:

clientes corporativos (uso muy alto → valiosos para Premium)

usuarios con anomalías técnicas (duración de llamadas = 120 minutos)

clientes mayores (edad > 60) que requieren soporte específico

datos erróneos que deben corregirse o eliminarse

Opciones de tratamiento:
Tratamiento	Cuándo usar
Mantener	si representan clientes VIP o comportamiento real
Winsorizar (cap)	si sesgan demasiado la distribución
Eliminar	si son imposibles (ej. duración 9999 min)



🔍 Hint

Crea una columna nueva:

user_profile['clean_minutes'] = np.where(user_profile['outlier_flag_minutes'], upper, user_profile['total_call_minutes'])


In [ ]:
# TODO: decide qué hacer con los outliers en total_call_minutes

# Ejemplo: winsorizar valores extremos
# user_profile['clean_minutes'] = np.where(
#     user_profile['outlier_flag_minutes'],
#     upper,
#     user_profile['total_call_minutes']
# )

# user_profile[['total_call_minutes', 'clean_minutes']].head()


✅ **Solución (instructor)**

In [ ]:
# Ejemplo de tratamiento: winsorización
user_profile['clean_minutes'] = np.where(
    user_profile['outlier_flag_minutes'],
    upper,
    user_profile['total_call_minutes']
)

print(user_profile[['total_call_minutes', 'clean_minutes']].head())


✍️ Comentario:

Escribe aquí un breve diagnóstico:

- ¿Qué columnas tienen outliers?

- ¿Cual es la razon del outlier?

---

## 7. Segmentación programática de clientes

🎯 **Objetivo:** crear una nueva columna `user_segment` usando reglas basadas en el comportamiento y la edad.

Ejemplo de reglas (puedes ajustarlas si quieres):
- `A_Mensajero`: más del 80% de eventos son textos.
- `B_Conversador`: más del 60% de eventos son llamadas.
- `D_Corporativo`: duración promedio de llamada > 20 minutos.
- `E_Senior`: edad > 60 y duración promedio de llamada > 8 minutos.
- `C_Mixto`: resto de usuarios.

Vas a:
- Crear una función `assign_segment(row)`.
- Usar `.apply(assign_segment, axis=1)` sobre `user_profile`.

### 🔍 Hint
- Calcula `pct_text = text_count / total_events` y `pct_call = call_count / total_events`.
- Maneja casos donde `total_events` es 0 o NaN.


In [ ]:
# TODO: definir la función de segmentación y aplicarla

# def assign_segment(row):
#     total = row['total_events']
#     call_count = row['call_count']
#     text_count = row['text_count']
#     age = row['age']
#     avg_call = row['avg_call_duration']
#
#     # Manejar total = 0
#     if pd.isna(total) or total == 0:
#         return 'Sin_uso'
#
#     pct_text = text_count / total
#     pct_call = call_count / total
#
#     # TODO: implementar reglas
#     # if ...:
#     #     return 'A_Mensajero'
#
#     return 'C_Mixto'

# user_profile['user_segment'] = user_profile.apply(assign_segment, axis=1)
# user_profile['user_segment'].value_counts()

✅ **Solución (instructor)**

In [ ]:
def assign_segment(row):
    total = row['total_events']
    call_count = row['call_count']
    text_count = row['text_count']
    age = row['age']
    avg_call = row['avg_call_duration']

    if pd.isna(total) or total == 0:
        return 'Sin_uso'

    pct_text = text_count / total if total else 0
    pct_call = call_count / total if total else 0

    if pct_text > 0.80:
        return 'A_Mensajero'
    if pct_call > 0.60:
        return 'B_Conversador'
    if (avg_call is not None) and (avg_call > 20):
        return 'D_Corporativo'
    if (age is not None) and (age > 60) and (avg_call is not None) and (avg_call > 8):
        return 'E_Senior'
    return 'C_Mixto'

user_profile['user_segment'] = user_profile.apply(assign_segment, axis=1)

print(user_profile['user_segment'].value_counts())

✍️ Comentario:

Escribe aquí una breve explicación sobre tu regla de negocios que has definido para segmentar los clientes.

---

## 8. Visualización de segmentos

🎯 **Objetivo:** entender visualmente cómo se comportan los segmentos.

Ideas de gráficos:
- Conteo de usuarios por segmento (barplot).
- Boxplot de `total_call_minutes` por segmento.
- Boxplot de `text_count` por segmento.

### 🔍 Hint
- `sns.countplot(x='user_segment', data=user_profile)`.
- Para boxplots: `sns.boxplot(x='user_segment', y='total_call_minutes', data=user_profile)`.


In [ ]:
# TODO: crear visualizaciones por segmento

# Conteo por segmento
# sns.countplot(x='user_segment', data=user_profile)
# plt.xticks(rotation=45)
# plt.title('Número de usuarios por segmento')
# plt.show()

# Boxplot de minutos de llamada por segmento
# sns.boxplot(x='user_segment', y='total_call_minutes', data=user_profile)
# plt.xticks(rotation=45)
# plt.title('Total de minutos de llamada por segmento')
# plt.ylim(0, 200)
# plt.show()

✅ **Solución (instructor)**

In [ ]:
sns.countplot(x='user_segment', data=user_profile)
plt.xticks(rotation=45)
plt.title('Número de usuarios por segmento')
plt.show()

sns.boxplot(x='user_segment', y='total_call_minutes', data=user_profile)
plt.xticks(rotation=45)
plt.title('Total de minutos de llamada por segmento')
plt.ylim(0, 200)
plt.show()

---

## 9. Insight ejecutivo para stakeholders

🎯 **Objetivo:** traducir tus hallazgos a lenguaje de negocio.

En 8–12 líneas, responde:
- ¿Qué segmentos identificaste y cómo se comportan?  
- ¿Qué segmentos parecen más valiosos para ConnectaTel?  
- ¿Qué patrones de outliers encontraste (clientes corporativos, uso extremo)?  
- ¿Qué recomendaciones harías sobre nuevos planes o ajustes a los planes actuales?

✍️ Escribe tu respuesta aquí en texto:


---

## 10. Cargar tu notebook y README a GitHub

🎯 **Objetivo:** entregar tu análisis de forma profesional y versionada.

Tienes dos opciones para subir este notebook y un README descriptivo a tu repositorio de GitHub.

### Opción A — Usando la interfaz de GitHub (UI)
1. Descarga este notebook (`File → Download .ipynb` si estás en Colab).  
2. Ve a tu repositorio en GitHub (por ejemplo: `telecom-analysis` o `sprint6-final-project`).  
3. Haz clic en **Add file → Upload files**.  
4. Sube tu notebook dentro de la carpeta `notebooks/`.  
5. Sube también un `README.md` que explique:
   - el objetivo del análisis,  
   - qué contienen los datasets y el notebook,  
   - cómo abrirlo (por ejemplo, en Google Colab),  
   - una breve guía de reproducción.

### Opción B — Usando Google Colab
1. Abre este notebook en Colab.  
2. Ve a **File → Save a copy in GitHub**.  
3. Elige el repositorio y la carpeta (por ejemplo `notebooks/`).  
4. Escribe un commit message claro, por ejemplo:  
   `feat: add final telecom usage analysis and segmentation`.  
5. Verifica en GitHub que el archivo quedó en la carpeta correcta y que el historial de commits se ve limpio.

> ✅ Con esto completas el ciclo: análisis → segmentación → documentación → versionamiento → entrega profesional.
